### 직방의 원룸 매물정보 수집
- 절차
    - 동이름 > 위도,경도
    - 위도,경도 > geohash(영역) 변환
    - geohash(영역) > 매물 아이디
    - 매물 아이디 > 매물 정보

In [7]:
import requests
import pandas as pd
import geohash2

In [ ]:
!pip install geohash2

In [2]:
# 1. 동이름 > 위도, 경도

In [12]:
addr = "강남구 역삼동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.495365142822266, 127.03306579589844)

In [3]:
# 2. 위도, 경도 > geohash

In [15]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydm6'

In [4]:
# 3. geohash > 매물 아이디

In [22]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
response = requests.get(url)
response

<Response [200]>

In [20]:
items = response.json()["items"]
len(items), items[:2]

(2344,
 [{'lat': 37.485141261794524, 'lng': 127.01296495741643, 'item_id': 32819635},
  {'lat': 37.485627384450254, 'lng': 127.0136283391101, 'item_id': 32838798}])

In [21]:
ids = [item["item_id"] for item in items]
ids[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

In [5]:
# 4. 매물 아이디 > 매물 정보

In [23]:
url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids[:900]
}
response = requests.post(url, params)
response

<Response [200]>

In [27]:
items = response.json()["items"]
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
df = pd.DataFrame(items)[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
898,32844154,월세,100,100,29.75,서울시 강남구 역삼동,9
899,32656329,전세,24000,0,59.50,서울시 강남구 역삼동,10


In [33]:
# max row, max column 설정
pd.options.display.max_columns, pd.options.display.max_rows

(50, 60)

In [31]:
pd.options.display.max_columns = 50

In [32]:
pd.DataFrame(items).tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
898,None,32844154,https://ic.zigbang.com/ic/items/32844154/1.jpg,월세,월세,100,100,29.75,"{'m2': 29.75, 'p': '9'}","{'m2': 29.75, 'p': '9'}",None,None,2,2,2,🟥🟦단기💖맛집🟦🟥 리모델링 주거/업무 겸용 원룸!,None,01,강남구 역삼동,"{'lat': 37.5016950823564, 'lng': 127.028856632...",False,True,원룸,[추천],서울시 강남구 역삼동,None,None,9,2022-08-01T18:43:35+09:00,False
899,None,32656329,https://ic.zigbang.com/ic/items/32656329/1.jpg,전세,전세,24000,0,59.50,"{'m2': 59.5, 'p': '18'}","{'m2': 34.71, 'p': '10.5'}",None,None,중,중,26,"◈전입가능◈위치,크기좋은 전세◈",None,01,강남구 역삼동,"{'lat': 37.501714354904976, 'lng': 127.0439070...",False,True,오피스텔,[추천],서울시 강남구 역삼동,None,None,10,2022-07-18T13:23:18+09:00,False


In [34]:
# 함수 만들기
def oneroom(addr):
    
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids[:900]
    }
    response = requests.post(url, params)
    
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns]

In [36]:
addr = "마포구 합정동"
df = oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
742,32862490,전세,43000,0,31.74,서울시 마포구 중동,5
743,32866975,월세,3000,90,49.59,서울시 마포구 중동,2


In [39]:
df_filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
95,32794268,월세,1000,90,59.50,서울시 마포구 합정동,2
96,32794303,월세,3000,80,59.50,서울시 마포구 합정동,2
97,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
98,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2
